# **Part 1**

In [1]:
import requests
import random

# Bitfinex API URL for order book depth (for the BTC-ETH trading pair)
url = "https://api-pub.bitfinex.com/v2/book/tETHBTC/P0"

# Send request to Bitfinex API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Separate bids and asks
    bids = [entry for entry in data if entry[2] > 0]  # Positive amount is a bid
    asks = [entry for entry in data if entry[2] < 0]  # Negative amount is an ask

    best_bid = bids[0][0]  # Get the best bid price
    best_ask = asks[0][0]  # Get the best ask price

    print(f"Best Bid: {best_bid}")
    print(f"Best Ask: {best_ask}")
else:
    print(f"Failed to fetch data from Bitfinex API: {response.status_code}")
    print(f"Error details: {response.text}")

Best Bid: 0.038971
Best Ask: 0.038982


In [2]:
def generate_order_book(best_bid, best_ask, num_orders=12):
    # Generate bid list, ensure the first order is the best bid
    bid_list = [(best_bid, round(random.uniform(0.01, 1), 6))]
    current_bid = best_bid
    for i in range(1, num_orders):
        price = round(current_bid - random.uniform(0.0001, 0.001), 6)  # Ensure decrement
        if price <= 0:
            price = round(random.uniform(0.001, best_bid), 6)
        quantity = round(random.uniform(0.01, 1), 6)
        bid_list.append((price, quantity))
        current_bid = price  # Update current bid price

    # Generate ask list, ensure the first order is the best ask
    ask_list = [(best_ask, round(random.uniform(0.01, 1), 6))]
    current_ask = best_ask
    for i in range(1, num_orders):
        price = round(current_ask + random.uniform(0.0001, 0.001), 6)  # Ensure increment
        quantity = round(random.uniform(0.01, 1), 6)
        ask_list.append((price, quantity))
        current_ask = price  # Update current ask price

    return bid_list, ask_list

In [3]:
bid_list, ask_list = generate_order_book(best_bid, best_ask)

# Output the generated order book
print("\nGenerated Bid List:")
for bid in bid_list:
    print(f"Price: {bid[0]}, Quantity: {bid[1]}")

print("\nGenerated Ask List:")
for ask in ask_list:
    print(f"Price: {ask[0]}, Quantity: {ask[1]}")


Generated Bid List:
Price: 0.038971, Quantity: 0.243255
Price: 0.038738, Quantity: 0.209365
Price: 0.038386, Quantity: 0.167026
Price: 0.037577, Quantity: 0.574573
Price: 0.03695, Quantity: 0.187521
Price: 0.036648, Quantity: 0.036182
Price: 0.03644, Quantity: 0.739896
Price: 0.035958, Quantity: 0.603361
Price: 0.035526, Quantity: 0.891913
Price: 0.034839, Quantity: 0.745508
Price: 0.034041, Quantity: 0.654644
Price: 0.033199, Quantity: 0.797466

Generated Ask List:
Price: 0.038982, Quantity: 0.489764
Price: 0.03953, Quantity: 0.043105
Price: 0.040322, Quantity: 0.514962
Price: 0.041272, Quantity: 0.877513
Price: 0.041533, Quantity: 0.623035
Price: 0.042052, Quantity: 0.664239
Price: 0.042329, Quantity: 0.347127
Price: 0.042496, Quantity: 0.418709
Price: 0.042819, Quantity: 0.446088
Price: 0.043065, Quantity: 0.437021
Price: 0.043248, Quantity: 0.613309
Price: 0.043449, Quantity: 0.319264


# **Part 2**

In [ ]:
import time
import requests
import random

# Function to get the latest bid and ask prices from the exchange
def get_latest_bid_ask():
    # This is where your API call happens to ensure you get the latest best bid and best ask
    url = "https://api-pub.bitfinex.com/v2/book/tETHBTC/P0"
    response = requests.get(url)
    data = response.json()
    best_bid = data[0][0]  # Assuming we got the latest best bid
    best_ask = data[-1][0]  # Assuming we got the latest best ask
    return best_bid, best_ask

# Function to generate the order book with constraints
def generate_order_book(best_bid, best_ask, num_orders=12):
    # Generate bid list, ensure the sum of (size * price) is less than 5
    bid_list = []
    ask_list = []

    current_bid = best_bid
    current_ask = best_ask

    bid_total = 0  # This will store the sum of (size * price) for bid list
    ask_total = 0  # This will store the sum of size for ask list

    # Generate bid list ensuring the sum(size * price) < 5
    for _ in range(num_orders):
        price = round(current_bid - random.uniform(0.0001, 0.001), 6)
        quantity = round(random.uniform(0.01, 1), 6)
        # Stop adding orders if the total sum exceeds 5
        if bid_total + (price * quantity) >= 5:
            break
        bid_total += (price * quantity)
        bid_list.append((price, quantity))
        current_bid = price  # Update current bid price

    # Generate ask list ensuring the sum(size) < 150
    for _ in range(num_orders):
        price = round(current_ask + random.uniform(0.0001, 0.001), 6)
        quantity = round(random.uniform(0.01, 1), 6)
        # Stop adding orders if the total sum exceeds 150
        if ask_total + quantity >= 150:
            break
        ask_total += quantity
        ask_list.append((price, quantity))
        current_ask = price  # Update current ask price

    return bid_list, ask_list, bid_total, ask_total

# Timer loop to update prices and generate the order book every 30 seconds
while True:
    best_bid, best_ask = get_latest_bid_ask()  # Fetch the latest prices
    bid_list, ask_list, bid_total, ask_total = generate_order_book(best_bid, best_ask)

    # Print out the generated bid list and ask list with total values
    print("Bid List:", bid_list)
    print("Ask List:", ask_list)
    print(f"Sum of (size * price) for bid list: {bid_total}")
    print(f"Sum of size for ask list: {ask_total}")

    # Wait 30 seconds before updating prices and regenerating the order book
    time.sleep(30)